---
# Hands-on session
# Solving a logistics distribution problem using mixed-integer programming and matheuristics (June, 2018)

Pedro Castellucci, The University of Sao Paulo and University of Melbourne

Alysson Costa, The University of Melbourne

---

This Notebook will guide us through an example of using matheuristics to solve a particular Mixed Integer Linear Programming model. We begin by implementing the model and then we explore some ideas related to [Local Branching](https://link.springer.com/article/10.1007/s10107-003-0395-5) and [Proximity Search](https://link.springer.com/article/10.1007/s10732-014-9266-x). Although we are focusing on a particular example, note that the ideas are generic enough to be applied to any optimisation problem defined as $Min\{c^T x: Ax \leq b, x \in \{0, 1\}^n\}$.


Without going into much detail about the model, it has the following input parameters:

- $C$: a set of consumers.
- $S$: a set of suppliers.
- $w$: a collaborative distribution centre (cross-dock).
- $N = S \cup C \cup \{w\}$: set of nodes in the network.
- $T$: a set of time periods.
- $d_{is}$: 1 if consumer $i$ has demand from supplier $s$, 0 otherwise.
- $c_{ij}$: cost of going from node $i$ to node $j$.

Also, the following binary variables were defined:

- $x_{ijst}$: indicates whether the truck from supplier $s$ goes from $i$ to $j$ at time period $t$.
- $y_{is}$: indicates whether consumer $i$ is visited by supplier $s$.
- $z_{iss'}$: indicates whether the demand of consumer $i$ from supplier $s$ is satisfied by the truck from supplier $s'$.

Thereby, we can define the following model for minimising the total distribution costs.

$\displaystyle \min \sum_{i \in N} \sum_{j \in N} \sum_{s \in S} \sum_{t \in T} c_{ij} x_{ijst}$

Subject to:

(We cannot leave any suppliers if  $t \neq 1$)

$\displaystyle
x_{sjst} = 0, \quad  j \in N, s \in S, t \in T, t \neq 1,$

(We cannot leave nodes that are not suppliers at $t = 1$)

$\displaystyle
x_{ijs1} = 0, \quad  i \in N \backslash \{s\}, j \in N, s \in S,$

(If we reach a consumer or the cross-dock at time period $t$, we leave it at $t+1$)

$\displaystyle
\sum_{i \in N} x_{ijst} = \sum_{i \in N} x_{jis,t+1}, \quad j \in C \cup \{w\}, s \in S, t \in T,$

(Each supplier visits a consumer at most once)

$\displaystyle
\sum_{i \in N} \sum_{t \in T} x_{ijst} \leq 1, \quad j \in N, s \in S,$

(The demand must be served by one of the suppliers)

$\displaystyle 
\sum_{j \in N} \sum_{t \in T} x_{jist} \geq d_{is} - \sum_{s' \in S, s' \neq s} z_{iss'}, \quad i \in C, s \in S,$

(A supplier $s'$ may respond to a demand for supplier $s \neq s'$ if the trucks from $s$ and $s'$ visit the cross-dock and $s'$ visits the consumer after visiting the cross-dock)

$\displaystyle
3z_{iss'} \leq \sum_{j \in C \cup \{s\}} \sum_{t \in T} x_{jwst} + \sum_{j \in C \cup \{s\}} \sum_{t \in T} x_{jws't} + y_{is'}, \quad i \in C, s \in S, s' \in S, s \neq s',$ 

(Whether a consumer is visited by a truck after the truck has visited the cross-dock ($y_{is}$))

$\displaystyle
\sum_{j \in C\cup \{s\}} \sum_{t \in T} t\ x_{jist} - \sum_{j \in C \cup \{s\} \cup \{w\}} \sum_{t \in T} t\ x_{jwst} \geq |T| (y_{is}-1), \quad i \in N, s \in S,$

(Ensuring that if a consumer $i$ is not visited by the truck from $s$ than $y_{is}$ = 0)

$\displaystyle
y_{is} \leq \sum_{j \in C \cup \{s\}} \sum_{t \in T} x_{jist}, \quad i \in C, s \in S.$

We need some data to play with. We will be using the problem described in file [data1.csv](data/data1.csv). The following figure shows the positions of suppliers (nodes 1 and 2, in red), consumers (nodes 3 to 12, in green) and the cross-dock (node 13, in blue). 

<img src="images/inputData.png" alt="Our problem">

Since we will be using JuMP/Julia to implement the model and GLPK as a MIP solver, we need to load the correspondent packages. Also, we are including some functions that will help us manipulate the data so we can focus on the matheuristics related aspects.

In [1]:
using JuMP
using GLPKMathProgInterface
include("src/readData.jl"); # Import function readData, used for reading input parameters, and others

In the following, we have a function that implements the model as presented previously. It returns the model and the $x_{ijst}$, $y_{is}$, $z_{iss'}$ variables.

In [2]:
function defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

    T = 1:length(nodes)  # Possible legs

    @variable(m, x[i=nodes, j=nodes, s=suppliers, t=T; edgeExists[s, i, j]], Bin)
    @variable(m, z[i=consumers, s=suppliers, sBar=suppliers; s != sBar], Bin)
    @variable(m, y[i=consumers, s=suppliers], Bin)

    @objective(m, Min, sum(cost[i, j]*x[i, j, s, t]
        for i in nodes, j in nodes, s in suppliers, t in T if edgeExists[s, i, j]))

    # We only leave suppliers at the first leg:
    @constraint(m, [j in nodes,
                    s in suppliers,
                    t in T; (t != 1) && (edgeExists[s, j, s])],
                    x[s, j, s, t] == 0)

    # We cannot leave any other node but suppliers in the first leg:
    @constraint(m, [i in nodes,
                    j in nodes,
                    s in suppliers; (i in suppliers) == false && (edgeExists[s, i, j])],
                    x[i, j, s, 1] == 0)

    # Flow balance constraints:
    @constraint(m, [s in suppliers, j in nodes, t in T[1:end-1]; j != s],
    sum(x[i, j, s, t] for i in nodes if edgeExists[s, i, j]) ==
    sum(x[j, i, s, t+1] for i in nodes if edgeExists[s, j, i]))

    # We only visit each node at most once with each supplier
    for s in suppliers, j in nodes
        @constraint(m, sum(x[i, j, s, t] for i in nodes, t in T if edgeExists[s, i, j]) <= 1)
    end

    # Demand constraints:
    @constraint(m, [i in consumers, s in suppliers],
        sum(x[j, i, s, t] for j in nodes, t in T if edgeExists[s, i, j]) >=
        demand[i, s] - sum(z[i, s, sBar] for sBar in suppliers if s != sBar))

    for i in consumers, s in suppliers
        @expression(m, sVisitsCD, sum(x[j, w, s, t] for j in nodes, t in T if edgeExists[s, j, w]))

        for sBar in suppliers
            if s != sBar
                @expression(m, sBarVisitsCD, 
                            sum(x[j, w, sBar, t] for j in nodes, t in T if edgeExists[sBar, j, w]))
                @constraint(m, 3*z[i, s, sBar] <= sVisitsCD + sBarVisitsCD + y[i, sBar])
            end
        end
    end

    @constraint(m, [i in consumers, s in suppliers],
        sum(t*x[j, i, s, t] for j in nodes, t in T if edgeExists[s, j, i]) -
        sum(t*x[j, w, s, t] for j in nodes, t in T if edgeExists[s, j, w]) >= length(T)*(y[i, s]-1))

    @constraint(m, [i in consumers, s in suppliers],
        y[i, s] <= sum(x[j, i, s, t] for j in nodes, t in T if edgeExists[s, j, i]))

    m, x, y, z
end

defineModel (generic function with 1 method)

Now, we are prepared to use GLPK/JuMP to solve our model.

In [3]:
filename = "data/data1.csv"  # The file with the input data

# We read the input data with the readData function
suppliers, consumers, warehouse, nodes, posX, posY, cost, demand = readData(filename)
w = warehouse[1]

# We need some preprocessing of the input parameters.
# After preprocessing we get the edgeExists map, 
# which tell us if a particular supplier, s, can traverse edge (i, j)
nodes, consumers, suppliers, edgeExists = preprocessing(nodes, suppliers, consumers, demand)

# Time Limit (tm_lim) is in mili-seconds:
m = Model(solver=GLPKSolverMIP(tm_lim=20000, msg_lev=2))

m, x, y, z = defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

# Suppress warnings because we like to live dangerously:
solve(m, suppress_warnings=true)

      0: obj =   0.000000000e+00 inf =   1.300e+01 (13)
    233: obj =   5.848602719e+02 inf =   1.662e-15 (0)
*   500: obj =   2.403272906e+02 inf =   2.721e-16 (759) 1
*   772: obj =   1.744989748e+02 inf =   0.000e+00 (0) 1
+   772: mip =     not found yet >=              -inf        (1; 0)
+  5239: >>>>>   4.896222297e+02 >=   2.427393813e+02  50.4% (162; 4)
+ 11853: >>>>>   4.729352578e+02 >=   2.603219018e+02  45.0% (255; 145)
+ 17674: mip =   4.729352578e+02 >=   2.690435296e+02  43.1% (429; 183)
+ 24593: mip =   4.729352578e+02 >=   2.773211854e+02  41.4% (616; 200)
+ 27499: mip =   4.729352578e+02 >=   2.809593647e+02  40.6% (685; 208)


:UserLimit

The *solve* method give us a status for the optimisation procedure. If we want the solution itself, we can use the *printSolution* method from the *readData.jl* file.

In [5]:
printSolution(m, suppliers, nodes, edgeExists)

579.1339215717855
1 1 13
1 13 3
1 3 6
1 6 7
1 7 1
2 2 13
2 13 10
2 10 8
2 8 12
2 12 11
2 11 9
2 9 4
2 4 5
2 5 2



The solution looks like the following (note that you might get a different solution when running in a different hardware. In that case, the figure will not reflect the solution).

<img src="images/modelSolution.png">

# [Local branching](https://link.springer.com/article/10.1007/s10107-003-0395-5)

Now, let us explore some ideas from *Local Branching* to solve our problem. First, we need an initial feasible solution, which we will get by trying to solve the model for a limited time. Then, we will add a constraint of the following type and reoptimise.

$$\sum_{\bar{x} \in \mathcal{X}^-} \bar{x} + \sum_{\bar{x} \in \mathcal{X}^+} (1 - \bar{x}) \leq k,$$

in which $\mathcal{X}^ -$ ($\mathcal{X}^+$) is the set of all variables in the problem that have a value of zero (one) in the first optimisation and $k$ is the "radius" of the neighbourhood.

For this, add the constraint described above after the first call of the *solve* method. Start with $k=10$ and check how different values affect the solution.

In [8]:
m = Model(solver=GLPKSolverMIP(tm_lim=10000, msg_lev=0))

m, x, y, z = defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

# This is the first optimisation:
solve(m, suppress_warnings=true)

println("Initial solution $(getobjectivevalue(m))") 

# We can use the separateZerosOnes function (readData.jl)
varZeros, varOnes = separateZerosOnes(m)

# Add the constraint here. You can iterate over varZeros (or varOnes)
# using "for var in varZeros". The general syntax for adding a constraint is:
# @constraint(model, expression <= k)

solve(m, suppress_warnings=true)

obj = getobjectivevalue(m)

if isnan(obj)
    println("\nCould not find a feasible solution within the time limit.") 
else
    println("\nSolution after reoptimisation: $obj")    
end

Initial solution 579.1339215717855

Solution after reoptimisation!: 548.7311820083075


For some values of $k$, the solver may not be able to find any feasible solution within the time limit. For handling such cases, it is possible to try a reoptimisation with smaller value of $k$ in the hope that the resulting problem is easier to solve. Let us add this rule for updating $k$.

In [9]:
m = Model(solver=GLPKSolverMIP(tm_lim=10000, msg_lev=0))

m, x, y, z = defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

# This is the first optimisation:
solve(m, suppress_warnings=true)

# Separating zeros and ones (readData.jl)
varZeros, varOnes = separateZerosOnes(m)

initialK = 25

@constraint(m, localBranchConstr,
        sum(var for var in varZeros) + sum(1 - var for var in varOnes) <= initialK)

solve(m, suppress_warnings=true)
obj = getobjectivevalue(m)

@show obj, initialK

iter = 1

# We can use isnan method to check if a solution is available.
while isnan(obj) && iter < 10
    # You may add the rule for updating the initialK variable here:
    
    # Changing the right-hand side of the constraint:
    JuMP.setRHS(localBranchConstr, initialK) 
    
    solve(m, suppress_warnings=true)
    
    obj = getobjectivevalue(m)
    @show obj, initialK
    
    iter += 1
end

(obj, initialK) = (NaN, 25)
(obj, initialK) = (NaN, 12.5)
(obj, initialK) = (740.3881660495049, 6.25)


Note that if the solver is able to find an optimal solution for a particular neighbourhood, it is possible to increase $k$ and reoptimise, hoping to find an even better solution for the problem. 

Besides changing the neighbourhood radius, another way to explore Local Branching is by changing the centre of the neighbourhood, i.e., the solution that we are using as a pivot for defining the neighbourhood. One implementation for this would be the following.

In [10]:
mOriginal = Model(solver=GLPKSolverMIP(tm_lim=10000, msg_lev=0))
mOriginal, x, y, z = defineModel(mOriginal, nodes, suppliers, consumers, demand, edgeExists)

# Now we are not modifying original model, i.e., 
# the model without the local branching constraint
m = copy(mOriginal)

# Printing the number of constraints for each model:
@show MathProgBase.numlinconstr(m), MathProgBase.numlinconstr(mOriginal)

# Getting the initial feasible solution:
solve(m, suppress_warnings=true)
@show getobjectivevalue(m)
println()  # Just adding a new line.

# This loops defines the number of iterations:
for i in 1:2
    
    # Saving the variables that have a value of one (readData.jl):
    xOnes, yOnes, zOnes = getOnes(m)

    # Be aware that copying the model is not efficient:
    m = copy(mOriginal)

    # Separating zeros and ones:
    varZeros, varOnes = separateZerosOnes(m, xOnes, yOnes, zOnes)
    
    # Adding the local branching constraint:
    @constraint(m, localBranchConstr,
            sum(var for var in varZeros) + sum(1 - var for var in varOnes) <= 10)

    # Printing the number of constraints for each model:
    @show MathProgBase.numlinconstr(m), MathProgBase.numlinconstr(mOriginal)
    
    # Solving with the local branching constraint:
    solve(m, suppress_warnings=true)
    
    @show getobjectivevalue(m)
    println()  # Just to adding a new line.
end

(MathProgBase.numlinconstr(m), MathProgBase.numlinconstr(mOriginal)) = (900, 900)
getobjectivevalue(m) = 579.1339215717855
(MathProgBase.numlinconstr(m), MathProgBase.numlinconstr(mOriginal)) = (901, 900)
getobjectivevalue(m) = 581.0173438221805
(MathProgBase.numlinconstr(m), MathProgBase.numlinconstr(mOriginal)) = (901, 900)
getobjectivevalue(m) = 550.6146042587021


Note that, without providing the previous solution to a current step, there is no guarantee that the current solution is not worse than the previous one. Also, you can combine the ideas of changing the neighbourhood radius and centre to build more sophisticated searching strategies.

# [Proximity search](https://link.springer.com/article/10.1007/s10732-014-9266-x)

The basic algorithm of the Proximity Search begins with finding an initial feasible solution ($\tilde{x}$). Here, we will use the model itself for finding this initial solution, but any ad-hoc heuristics would be enough. Then, we need to define the cutoff tolerance $\theta$, this parameter will be used in a constraint to be added to the model. The constraint looks like the following (assuming a minimisation problem).

$$f(x) \leq f(\tilde{x}) - \theta.$$

Before reoptimising the problem, we change its objective function to:

$$Min\ \sum_{\bar{x} \in \mathcal{X}^-} \bar{x} + \sum_{\bar{x} \in \mathcal{X}^+} (1 - \bar{x}),$$

in which $\mathcal{X}^ -$ ($\mathcal{X}^+$) is the set of all variables which have a value of zero (one) in the previous optimisation procedure. Note that this expression is the left-hand side of the constraint used in Local Branching.

Let us implement an iteration of the Proximity Search for our problem, using $\theta = 1$.

In [12]:
m = Model(solver=GLPKSolverMIP(tm_lim=10000, msg_lev=0))

m, x, y, z = defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

# Initial optimisation:
solve(m, suppress_warnings=true)

# Since we will be changing the objective function for the solver,
# we need to keep track of the original objective function:
T = 1:length(nodes)
@expression(m, objExpr, 
    sum(cost[i, j]*x[i, j, s, t] for i in nodes, 
        j in nodes,
        s in suppliers,
        t in T if edgeExists[s, i, j]))
                
@show getvalue(objExpr)
                
varZeros, varOnes = separateZerosOnes(m)

# The objective function can be redefined using
# @objective(m, Min, expression):
                
solve(m, suppress_warnings=true)

@show getvalue(objExpr);

getvalue(objExpr) = 579.1339215717857
getvalue(objExpr) = 577.4900375773768


Once we find a feasible solution, we can look for better ones just repeating the same procedure. If we do not change cutoff parameter $\theta$, at every iteration, each added constraint dominates the previous one, so there is no need to handle the elimination of constraints from the model. 

In [4]:
m = Model(solver=GLPKSolverMIP(tm_lim=10000, msg_lev=0))

m, x, y, z = defineModel(m, nodes, suppliers, consumers, demand, edgeExists)

# Initial optimisation:
solve(m, suppress_warnings=true)

@show getobjectivevalue(m)

# We want to keep track of the original objective function:
T = 1:length(nodes)
@expression(m, objExpr, 
    sum(cost[i, j]*x[i, j, s, t] for i in nodes, j in nodes, s in suppliers, t in T if edgeExists[s, i, j]))

# Loop defining the iterations
for i in 1:7
    
    # Separating variables:
    varZeros, varOnes = separateZerosOnes(m)

    # Redefining the objective function
    @objective(m, Min, sum(var for var in varZeros) + sum(1 - var for var in varOnes))
   
    # Adding the Proximity Search constraint:
    @constraint(m, objExpr <= getvalue(objExpr) - 1)

    solve(m, suppress_warnings=true)

    # If we cannnot find a feasible solution, we stop!
    if isnan(getobjectivevalue(m))
        break
    end

    # Getting value of the original objective function:
    @show i, getvalue(objExpr)
end  

# Remember that the objective function of the model is not the original one.
printSolution(m, suppliers, nodes, edgeExists)

getobjectivevalue(m) = 579.1339215717855
(i, getvalue(objExpr)) = (1, 577.4900375773768)
(i, getvalue(objExpr)) = (2, 554.6787904849061)
(i, getvalue(objExpr)) = (3, 547.7768699326865)
(i, getvalue(objExpr)) = (4, 526.7302762448531)
(i, getvalue(objExpr)) = (5, 520.9052411226269)
(i, getvalue(objExpr)) = (6, 519.7419488161277)
(i, getvalue(objExpr)) = (7, 513.952225256473)
18.0
1 1 13
1 13 6
1 6 7
1 7 1
2 2 13
2 13 3
2 3 5
2 5 4
2 4 9
2 9 11
2 11 8
2 8 12
2 12 10
2 10 2



Let us see what this solution looks like

<img src="images/proximitySearchSolution.png">

So far, we are using $\theta = 1$. Try other values of $\theta$ and see how they affect the evolution of the objective function. Then, define rules for modyfing $\theta$ dynamically.

This Notebook is just an example of exploring some ideas from Local Branching and Proximity Search. For detailed information on both methods, including a computational performance analysis, the reader is referred to the original papers:

- Fischetti, Matteo, and Andrea Lodi. "Local branching." Mathematical programming 98.1-3 (2003): 23-47.
- Fischetti, Matteo, and Michele Monaci. "Proximity search for 0-1 mixed-integer convex programming." Journal of Heuristics 20.6 (2014): 709-731.

For more information on JuMP/Julia check the [official documentation](http://jump.readthedocs.io/en/latest/).